# Chutes and Ladders

I read an article by Joel Smith about probabilities related to the popular children's board game, Chutes and Ladders. He specifically looked at how many turns it takes to complete a game for a given number of players. To perform this analysis, Smith used Markov Chains and Minitab software. I am using Monte Carlo simulation and Python to see how closely I can get to his results.

Smith, J. (2016, April 01). What Are the Odds? Chutes and Ladders. Retrieved July 10, 2017, from http://blog.minitab.com/blog/fun-with-statistics/what-are-the-odds-chutes-and-ladders

In [1]:
"""Simulate games of Chutes and Ladders."""

import random
import pandas as pd
import numpy as np

# Set random number generator seed to reproduce results
random.seed(9655)

## Movement on the Board

I first need a function for how pieces move on the board. The player first spins to see how many spaces to move. Normally, the player just moves that many spaces, but if the player gets a chute or ladder, they get moved to a predetermined space. Finally, if the resulting space is past the final spot (100), then the player does not move. 

In [2]:
def move(start, spin):
    """Move based on starting position and spinner value."""
    chutes_ladders = {1: 38, 4: 14, 9: 31, 16: 6, 21: 42, 28: 84, 36: 44,
                      48: 26, 49: 11, 51: 67, 56: 53, 62: 19, 64: 60, 71: 91,
                      80: 100, 87: 24, 93: 73, 95: 75, 98: 78}

    new_pos = start + spin
    try:
        final_pos = chutes_ladders[new_pos]
    except KeyError:
        if new_pos > 100:
            final_pos = start
        else:
            final_pos = new_pos
    return final_pos

## Simulating a Game

Next, I need a function to simulate a single game and return the number of turns required to complete it. The first step is to place all of the pieces at the starting spot. After that, I loop through turns until a player wins using random number generation. It is possible although unlikely for the game to keep going in an endless loop. To prevent this, I have capped the number of turns at 1,000.

In [3]:
def sim_game(players, start=0, slices=6):
    """Simulate a single game of Chutes and Ladders.

    Args:
        - players (int): the number of players (2-4 recommended).
        - start (int): starting square (default 0).
        - slices (int): number of spinner slices (default 6).

    Returns:
        number of turns needed to complete the game.

    """
    # Starting state at the starting square
    game_state = {}
    turn = 0
    exitflag = False
    for player in range(players):
        game_state[player] = start

    # Go through turns with a maximum number of 1000 turns
    while True:
        turn += 1
        if turn == 1000:
            break
        for player in range(players):
            game_state[player] = move(game_state[player],
                                      random.randrange(1, slices+1))
            if game_state[player] == 100:
                exitflag = True
        if exitflag:
            break
    return turn

## Simulate Many Games

I want to compare the point estimates shown in the article for 1 to 4 players. To approximate the results with simulation, I will use 100,000 iterations. I do not expect the simulated results to match the theoretical exactly but it should be close. As the number of simulations increase, then the difference should decrease.

In [4]:
def sim_games(players, sims=100000):
    """Simulate many games for a given number of players.

    Args:
        - players (int): the number of players (2-4 recommended).
        - sims (int): the number of simulated games (default 100,000)

    Returns:
        summary statistics of the number of turns to complete a game.

    """
    # Generate DataFrame of simulated games
    games = {}
    for i in range(sims):
        games[i] = sim_game(players)
    games_df = pd.DataFrame.from_dict(games, orient='index')
    games_df.columns = ['Turns']

    # Calculate summary statistics
    min_turns = games_df['Turns'].min()
    min_turns_prob = len(games_df[games_df['Turns'] == min_turns])/sims
    q90 = int(np.ceil(games_df.quantile(0.9).values[0]))
    q95 = int(np.ceil(games_df.quantile(0.95).values[0]))
    q99 = int(np.ceil(games_df.quantile(0.99).values[0]))
    med_turns = games_df['Turns'].median()
    mean_turns = games_df['Turns'].mean()

    # Print summary statistics
    print(f"\nFor {players:d} Player(s):")
    print(f"Minimum of {min_turns:d} turns to complete the game")
    print(f"With a probability of {min_turns_prob:.4f}")
    print(f"90% of player-games will finish before turn {q90:d}")
    print(f"95% of player-games will finish before turn {q95:d}")
    print(f"99% of player-games will finish before turn {q99:d}")
    print(f"Median of {med_turns:.4f} turns to complete the game")
    print(f"Mean of {mean_turns:.4f} turns to complete the game")

    return games_df

In [5]:
if __name__ == '__main__':
    sim_games(1)
    sim_games(2)
    sim_games(3)
    sim_games(4)


For 1 Player(s):
Minimum of 7 turns to complete the game
With a probability of 0.0015
90% of player-games will finish before turn 73
95% of player-games will finish before turn 90
99% of player-games will finish before turn 128
Median of 33.0000 turns to complete the game
Mean of 39.4923 turns to complete the game

For 2 Player(s):
Minimum of 7 turns to complete the game
With a probability of 0.0032
90% of player-games will finish before turn 44
95% of player-games will finish before turn 53
99% of player-games will finish before turn 73
Median of 23.0000 turns to complete the game
Mean of 26.5206 turns to complete the game

For 3 Player(s):
Minimum of 7 turns to complete the game
With a probability of 0.0048
90% of player-games will finish before turn 35
95% of player-games will finish before turn 40
99% of player-games will finish before turn 53
Median of 20.0000 turns to complete the game
Mean of 21.7445 turns to complete the game

For 4 Player(s):
Minimum of 7 turns to complete th

## Conclusions

My results are close to those in the article. I was surprised that even with 100,000 simulations, the mean was not closer to the Markov Chain results. My next move should be learning how to replicate the Markov Chain results themselves. 

Like Smith says, there is really no in-game advantage to understanding the statistics only that adding more players speeds up the game. 